# Example: How to Download and Start Processing Sentinel-1 Data

### In this demo we will walk you through the steps of:

(1) Searching for and downloading Sentinel-1 SLC data

(2) Obtaining the proper supporting files for data processing (e.g. orbit files, DEM grids)

(3) Preparing to process your Sentinel-1 images

(4) Processing your images

(5) Viewing and interpreting the interferogram (after processing is complete)

These steps, as presented in this notebook, should be applicable to most study areas, though remember that the Sentinel-1 satellite is a C-Band instrument, so data in heavily vegetated areas should be used with care.

##### Additionally, we will refer you to other prepared GMTSAR examples (https://topex.ucsd.edu/gmtsar/downloads/), that have undergone steps (1), (2), and (3) already, so you can explore both other examples of Sentinel-1 data, as well as other examples using data from other SAR satellite sources.

Let's get started!




## (1) Searching for and downloading Sentinel-1 SLC Data (manually)

To download Sentinel-1 data using a manual web-viewer, you can either go to the: 

Alaska Satellite Facility (ASF): https://search.asf.alaska.edu/#/

or visit the European Space Agency (ESA) Copernicus Open Access Hub: https://scihub.copernicus.eu/dhus/#/home 

##### NOTE: You will need to make an account with ASF and/or Copernicus to actually download data (ASF will prompt you to enter your secure credentials before a data download, and Copernicus will ask you before you start your search)

---
In this demo, we will use ASF, but the data search functions are similar for each site. If you click on the ASF link above it will take you to a webpage that looks like this:

![01_ASF_splashpage](PNGS/01_ASF_splashpage.png)





### Identify your study area and draw a box to start the search

Today we are going to look at interseismic deformation from Sentinel-1 data around the southern Los Angeles area, so we can zoom into Los Angeles, and draw a box around the area we're interested in. Your cursor should already be a blue dot that you can click to form a box, but if not, just the draw a polygon or draw a box buttons in the toolbar:

![02_ASF_LAbox](PNGS/02_ASF_LAbox2.png)


### Next, set search parameters

The next thing we need to do is filter the 23,538 files that match our search area (see the number next to the "UPDATE" button at the top?). Click on the "Filters" button next to the UPDATE button at the top of the page to bring up the following:

![02_ASF_searchbox](PNGS/03_ASF_searchbox.png)

We are going to look for data in May and June of 2015, so we will chose a start date of 5/15/2015 and an end date of 06/10/2015. If you are looking to process a single pair of images, we'd recommend starting with two dates that are relatively close together in time, as that will increase the likelihood of good coherence between the two. 

We will use these filters from the drawdown menus to search for SLC TOPS mode (IW) data:

FILE TYPE: `L1 Single Look Complex (SLC)`

BEAM MODE: `IW`

POLARIZATION: `VV` and `VV+VH`

DIRECTION: `Ascending`

SUBTYPE: either will do, so we leave it blank (this is whether you want satellite S1A or S1B)

PATH START/END: If you know exactly what number of Sentinel-1 track of data you want, put that here. 

FRAME START/END: If you know exactly what Frame number in what track you want, put that here.



![04_ASF_params](PNGS/04_ASF_params.png)


### Check out results
Check out how many files we've filtered to! Let's see what those look like when we click "UPDATE":


![05_ASF_results](PNGS/05_ASF_results.png)


### Choosing SLC images to download

Now that we've filtered our results, we need to pick our two images to download and interfere! Since our study area is well covered by the frame highlighted in red in the above image, we only need to download data from one track. If your study area has split coverage from two tracks, we'd recommend picking one track and processing that pair to start with.

In our case, this track, or path, is track 064, which you can find if you expand the Scene Detail tool bar (see below) and read the detail info about this image. The highlighted image is immediately selected for you in the list, so you can easily choose and download what you want. This image selected was collected on 05/26/2015, and since we want the next image that's been collected closest in time, we can either scroll through and find the next image that also covers this same spatial area (looks like the image taken on 06/07/2015) 

![06_ASF_toolbar](PNGS/06_ASF_toolbar.png)



#### ASF's Baseline Tool

OR if you're feeling adventurous, you can use ASF's baseline tool by selecting one image you like, and then clicking on the `Baseline` button at the bottom of the screen. This will calculate and plot the perpendicular baseline and the temporal baseline of all images in the approximate location of that one first image. This makes it easy to find images that cover the same spatial area and close in time to the signal you are trying to investigate. From this plot and list, we can see that, indeed, the next closest image in time is the image collected on 06/07/2015.

![07_ASF_baseline](PNGS/07_ASF_baseline.png)

### Downloading Data

At this point, we've identified two images we want to download and process (the SLC image from 05/26/2015 and the image from 06/07/2015). To download these, we can click the little "shopping cart" button next to the image name, and that image will essentially be added to our shopping cart, accessed at the "Downloads" button at the top of the page, where we can then click the "Download" cloud shaped button. 

![08_ASF_download](PNGS/08_ASF_download.png)

However, to download data, you need to be signed in, so it may prompt you to do so at this time. 

NOTE: Because these tend to be very large files (e.g. 2-3 GB per SLC file), they make take a little while to download, depending on your internet connection. 

Once you have downloaded your SLC data, move them into a new directory and unzip them. Once unzipped, each of the data files will have a file name ending with `*.SAFE` .

In [3]:
# Let's look at these files; If running this on your local machine, 
# make sure this notebook is in the same directory as your data files
import os

! ls S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE


S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE-report-20150526T065857.pdf
annotation
manifest.safe
measurement
preview
support


We can see that in the `*.SAFE` directory there are more directories. The `measurement` directory contains the actual data files, in `*.tiff` format. The `annotation` directory contains `*.xml` files that will be important when co-registering our two images together to make an interferogram. The `preview` directory contains useful user maps, included a Google Earth `map_overlay.kml` file and a `quick-look.png`. From this `quick-look.png` we can see our study area, with the areas of Rancho Palos Verdes (top blue circle) and La Jolla (bottom blue circle) marked for reference:

![09_SAFEfile](PNGS/09_SAFE_overview.png)

Notice the three different subswaths of collected data from left to right. This is a key feature of TOPS-mode SAR data. We will process each one of these three subswaths separately, and then merge them after we calculate their interferograms. Addionally, one can choose to only process one or two of the subswaths alone. This is often done if one of your data file's subswatchs is entirely made up of water, to avoid processing the area of bad correlation.


#### What's with the long names?!

Yes, the SLC data file names are very long, but there is useful information in the file name, specifically the collection time of the image. Here is a helpful guide from the ASF website: https://asf.alaska.edu/data-sets/sar-data-sets/sentinel-1/sentinel-1-data-and-imagery/

![10_SAFE_name](PNGS/10_SAFE_name.png)

## (2) Obtaining the proper supporting files for data processing (e.g. orbit files, DEM grids)

In order to process these two SLC images and make interferograms, we will need a few supporting data files. The first of these types of files is an orbit file, which contains all the orbit information for the satellite that was collecting the image (i.e. the exact location of the satellite and it's trajectory). The second type of file we'll need is a DEM (Digital Elevation Model) grid, which will help us to correct for topographic phase effects and help us geocode and project our final results into latitude/longitude coordinates.

### Orbit Files

Orbit files are currently available through ESA's Precise Orbit Determination (POD) Hub: https://scihub.copernicus.eu/gnss/#/home . However, finding the *correct* orbit file that matches your SLC data file can be difficult, mainly because the file names are long, filled with numerical characters, and easy to misread. 

We have recently developed a GMTSAR script called `download_sentinel_orbits.csh` (or `download_sentinel_orbits_linux.csh` for LINUX users) that will download these orbit files for you, thereby saving you the pain of having to match dates from file names perfectly yourself. Let's look at the usage of this script:



In [4]:
# To go download orbits using download_sentinel_orbits.csh or download_sentinel_orbits_linux.csh
# please make a list of your *.SAFE files with the absolute path to their location on your machine
import os

! download_sentinel_orbits.csh



Usage: download_sentinel_orbits.csh safefilelist mode
  Downloads precise or restituted orbits for specific Sentinel-1 *.SAFE data files  

safefilelist:
    absolutepathto/filename1.SAFE
    absolutepathto/filename2.SAFE
    ......
mode:
    mode 1 = precise orbits (POEORB)
            (most users should choose precise orbits)
    mode 2 = temporary (restituted) orbits (RESORB)
            (only recent data (~last couple weeks) requires restituted
            orbits, because precise orbits are not yet finalized)

Example: download_sentinel_orbits.csh SAFEfile.list 1

Note: 
  (1) Files listed in safefilelist should be the .SAFE directory with absolute path.



Most of the time we want precise orbit files, not restituted orbit files, because precise orbit files have the finalized orbital trajectory information. If we run this, this script should download two files that end with `*.EOF`. 

NOTE: This script relies on the utility `wget` . If your machine does not have this utility, it will spit out an error like: `wget: command not found`. You will need to install `wget` to run this properly.

In [6]:
! ls *.EOF

S1A_OPER_AUX_POEORB_OPOD_20210307T064730_V20150525T225944_20150527T005944.EOF
S1A_OPER_AUX_POEORB_OPOD_20210307T104652_V20150606T225944_20150608T005944.EOF


And now we have our orbit files!

### DEM File

Next, we need to download our DEM that covers our study area, and in particular the area covered by our SAR images. We can download DEM a rectangle at a time from the website:

https://topex.ucsd.edu/gmtsar/demgen/

but we'll need to know the latitude and longitude boundaries of a box that entirely captures our SAR scene. To do this, we turn to Google Earth. We can plot the `map_overlay.kml` file from our `S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE/preview` directory in Google Earth to see it's extent in latitude/longitude:

![11_GE1](PNGS/11_GE_overlay.png)


As you can see the actual collected image is slightly sideways when projected into latitutde/longitude, but to more easily see the boundaries we'll need to use, we plot a white box behind the image. We plot two pins, and from those pins (or by hovering your cursor over the point and looking at the bottom bar where it reports latitude/longitude), you can grab the necessary lat/long boundaries.

![12_GE2](PNGS/12_GE_overlay2.png)

#### Download the DEM

Once, we have the boundary coordinates from Google Earth, then we can plug them in to the DEM generator online (https://topex.ucsd.edu/gmtsar/demgen/), and download our DEM file. The downloaded file will need to be unzipped, and inside the directory will be a file called `dem.grd`. We will want this file for processing.

![13_DEM](PNGS/13_DEM.png)


## (3) Preparing to process your Sentinel-1 images

Now, we're ready to set up the directory structure for processing our two SAR images and creating an interferogram. First, if you haven't made a working directory yet, make one now. We'll call ours `LA_Example`. Inside this working directory, make two more directories, one called `raw` and one called `topo`. 

In [ ]:
# For example:

#! mkdir LA_Example
#! cd LA_Example
#! mkdir raw
#! mkdir topo

In the `raw/` directory, place your two `*.SAFE` data file directories as well as your two `*.EOF` orbit files. 

In the `topo/` directory, place your `dem.grd` file. 

In [10]:
# let's check

! echo "Check raw/ first..."
! ls LA_Example/raw

! echo " "

! echo "Check topo/ next..."
! ls LA_Example/topo



Check raw/ first...
S1A_IW_SLC__1SDV_20150607T014936_20150607T015003_006261_00832E_3626.SAFE
S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE
S1A_OPER_AUX_POEORB_OPOD_20210307T064730_V20150525T225944_20150527T005944.EOF
S1A_OPER_AUX_POEORB_OPOD_20210307T104652_V20150606T225944_20150608T005944.EOF
 
Check topo/ next...
dem.grd


Now everything should be in the right place to start processing!

## (4) Processing your Sentinel-1 images

To process these two images we will be using a processing script called `p2p_S1_TOPS_Frame.csh`. One can also check out the universal script called `p2p_processing.csh` which can work with a variety of different satellite data types. 

When typing out the command to run either `p2p_S1_TOPS_Frame.csh` or `p2p_processing.csh`, make sure to copy and paste the long file names to avoid typos. In addition, make sure to pair the correct `*.SAFE` file with the correct orbit file, based on the dates of collection (e.g. file `S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE` should be paired with file `S1A_OPER_AUX_POEORB_OPOD_20210307T064730_V20150525T225944_20150527T005944.EOF` because the date of collection (2015/05/26) matches the date of the orbit file period (`S1A_*_V20150525T225944_20150527T005944*.EOF` or 2015/05/25 - 2015/05/27, the dates and times specifed after the capital letter "V" )

##### NOTE: Because this processing will likely take a couple hours, we do not actually run it as part of this notebook. However, the commands are written below for clarity and/or copying/pasting.

In [ ]:
# FROM THE README.txt FILE:
#  This is an example script to process a swath of TOPS mode SAR data collected by the Sentinel 1A satellite.
#  It will process every subswath and merge the resulting interferograms.
#
#  The final results may be found in the merge folder.
#  Interferograms for each individual subswath may be found in the F1, F2, and F3 folders.
#
# make a standard config file
#
! pop_config.csh S1_TOPS > config.txt
#
#   make a merged interferogram
#
! p2p_S1_TOPS_Frame.csh S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE S1A_OPER_AUX_POEORB_OPOD_20210307T064730_V20150525T225944_20150527T005944.EOF S1A_IW_SLC__1SDV_20150607T014936_20150607T015003_006261_00832E_3626.SAFE S1A_OPER_AUX_POEORB_OPOD_20210307T104652_V20150606T225944_20150608T005944.EOF config.txt vv 0
#
#
#  The final argument of the script is a processing factor that governs whether the subswaths will be processed sequentially or in parallel.
#  A processing factor of 1 will have the script process the subswaths in parallel.
#  A processing factor of 0 will have the script process the subswaths sequentially. This is better for computers less memory, but takes longer to executes.

If you want to delve into exactly what the processing workflow is for GMTSAR, please see the Jupyter Notebook `gmtsar_rs2_demo.ipynb` on the GMTSAR Short Course 2022 Github page: https://github.com/gmtsar/2022-unavco-course-gmtsar


## (5) Viewing and interpreting the interferogram (after processing is complete)

Okay, your processing ran for a couple hours, and now it's time to see what it did! The main, merged results for TOPS-mode interferograms will be in the `merge/` directory of our example directory, so let's see what is in there:

In [11]:
! ls LA_Example/merge

S1_20150526_014935_F1.LED gmt.history               phasefilt_mask.grd
S1_20150526_014935_F1.PRM mask.grd                  phasefilt_mask_ll.grd
config.txt                mask2.grd                 phasefilt_mask_ll.kml
corr.cpt                  merge_log                 phasefilt_mask_ll.png
corr.grd                  merge_log_corr            raln.grd
corr_ll.grd               merge_log_mask            ralt.grd
corr_ll.kml               phase.cpt                 tmp.filelist
corr_ll.png               phasefilt.grd             tmp_corrlist
dem.grd                   phasefilt_ll.grd          tmp_masklist
gauss_200                 phasefilt_ll.kml          tmp_phaselist
gmt.conf                  phasefilt_ll.png          trans.dat


There are a number of files in here, but the two main things we want to look at are `corr.grd` and its derivatives and `phase.grd` and its derivatives. The `corr.grd` is the correlation between the two images, while the `phase.grd` is the actual wrapped interferogram we have calculated. The file `phasefilt.grd` is a filtered wrapped interferogram that has undergone additional processing to ensure the signal is better recovered. 

All files with the ending `*_ll.*` means that those files have been projected to latitude/longitude coordinates, and will be far easier to look at and interpret than the files in radar coordinates. Let's look at `phasefilt_mask_ll.png`:

![14_phase](PNGS/14_phasefilt.png)

This interferogram shows a typical mix of atmospheric effects with interseismic deformation. Since these images are only twelve days apart, the amount of interseismic deformation is quite low, and is likely buried by the larger effects of the troposphere and/or ionosphere. 

## Time to play!

This is how you can set up any set of interferogram pairs for processing. We encourage you to try to download and set up your own examples with your particular study areas. 

If you want to see more examples that are already prepared for you, check out the "Sample InSAR data" section of the GMTSAR website:

https://topex.ucsd.edu/gmtsar/downloads/

Here you will find examples from all over the world, using multiple different satellites' data, that are ready to run. To see how exactly to run each of these examples, please read the README.txt files that are stored inside the main directory, after you download it. 

### Questions? Contact your breakout group instructor.